### **Simple Text Blob**


In [1]:

import pandas as pd
import numpy as np
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics as skm
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

os.chdir('/content/gdrive/MyDrive/Applied DL/Phase1_Models/Data')
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive/.shortcut-targets-by-id/1XX_-wjbH0kCJCaEP0RyTNLDgw2LzvMzw/Applied DL/Phase1_Models/Data'

In [3]:
X_train=pd.read_csv('X_train.csv').squeeze()
X_test=pd.read_csv('X_test.csv').squeeze()
y_train=pd.read_csv('y_train.csv').squeeze()
y_test=pd.read_csv('y_test.csv').squeeze()


In [11]:
X_train.head()

0    The Axion unit took eight hours to charge out ...
1    This screen protector does a great job of prot...
2    We have an older model Nuvi that we are too ch...
3    The discs work flawlessly...not a single coast...
4    Good quality case that protects your External ...
Name: review, dtype: object

<h4>import BERT model to get embeding vectors</h4>

In [5]:
#performing one hot encoding for output. 
dum_df = pd.get_dummies(y_train)
y_train = np.asarray(dum_df) #converting onehot encoded output variable as needed for softmax layer

In [37]:
dum_df = pd.get_dummies(y_test)
y_test = np.asarray(dum_df)

In [62]:
y_test.shape

(12000, 3)

Data Cleaning

In [48]:

import regex as re
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the review column
cleaned_reviews = X_train.apply(clean)
cleaned_reviews.head()

cleaned_reviews_test = X_test.apply(clean)

In [49]:
cleaned_reviews_test.shape

(12000,)

In [16]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Next two steps takes time


In [ ]:
# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

cleaned_reviews_2 = cleaned_reviews.apply(token_stop_pos)
cleaned_reviews_2.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
      if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
      else:
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

cleaned_reviews_3 = cleaned_reviews_2.apply(lemmatize)
cleaned_reviews_3.head()

Text Blob

In [21]:
from textblob import TextBlob
# function to calculate subjectivity
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity
    # function to calculate polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [50]:
fin_data = pd.DataFrame(columns = ['cleaned_reviews'])
fin_data['cleaned_reviews'] = cleaned_reviews

fin_data_test = pd.DataFrame(columns = ['cleaned_reviews'])
fin_data_test['cleaned_reviews'] = cleaned_reviews_test

Sentiment Classification

In [52]:

fin_data['Polarity'] = fin_data['cleaned_reviews'].apply(getPolarity) 
fin_data['Analysis'] = fin_data['Polarity'].apply(analysis)
fin_data.head()


fin_data_test['Polarity'] = fin_data_test['cleaned_reviews'].apply(getPolarity) 
fin_data_test['Analysis'] = fin_data_test['Polarity'].apply(analysis)


In [53]:
fin_data['Analysis'].value_counts()

Positive    38391
Negative     8284
Neutral      1325
Name: Analysis, dtype: int64

In [54]:
fin_data_test['Analysis'].value_counts()

Positive    9542
Negative    2096
Neutral      362
Name: Analysis, dtype: int64

In [63]:
fin_data['Numeric_Analysis'] = fin_data['Analysis'].replace(['Positive', 'Neutral','Negative'],[2,0,1])
fin_data_test['Numeric_Analysis'] = fin_data_test['Analysis'].replace(['Positive', 'Neutral','Negative'],[2,0,1])

In [65]:
fin_data_test.head()

,cleaned_reviews,Polarity,Analysis,Numeric_Analysis
0,I got these for my laptop so I could connect i...,0.065079,Positive,2
1,This is a finely made wide angle prime one of ...,0.142474,Positive,2
2,Some have complained about signal loss That s ...,0.090944,Positive,2
3,I admit that what made me buy this keyboard wa...,0.122071,Positive,2
4,Having seen and tried my son in law s MoKo cas...,0.212143,Positive,2


In [66]:
dum_df_2 = pd.get_dummies(fin_data['Numeric_Analysis'])
dum_df_2 = np.asarray(dum_df_2) 

dum_df_test = pd.get_dummies(fin_data_test['Numeric_Analysis'])
dum_df_test = np.asarray(dum_df_test) 



In [67]:
dum_df_test.shape

(12000, 3)

Train and Test accuracy

In [68]:

print( skm.classification_report(argmax(y_train, axis=1), argmax(dum_df_2, axis=1)))
# y_train_p = model.predict(X_train) #train prediction

              precision    recall  f1-score   support

           0       0.50      0.04      0.08     16000
           1       0.25      0.13      0.17     16000
           2       0.39      0.93      0.55     16000

    accuracy                           0.37     48000
   macro avg       0.38      0.37      0.27     48000
weighted avg       0.38      0.37      0.27     48000



In [69]:
print( skm.classification_report(argmax(y_test, axis=1), argmax(dum_df_test, axis=1)))


              precision    recall  f1-score   support

           0       0.51      0.05      0.08      4000
           1       0.25      0.13      0.18      4000
           2       0.39      0.92      0.54      4000

    accuracy                           0.37     12000
   macro avg       0.38      0.37      0.27     12000
weighted avg       0.38      0.37      0.27     12000

